<a href="https://colab.research.google.com/github/LCaravaggio/politext/blob/snowball/GS_v0_2_con_vocab_allbase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carga datos

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import json

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

drive.mount('/content/drive', force_remount=True)
with open("/content/drive/My Drive/kaggle.json", 'r') as f:
    api_token= json.load(f)

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d leonardocaravaggio/base-diputados

Mounted at /content/drive
 97% 673M/696M [00:13<00:00, 78.0MB/s]
100% 696M/696M [00:13<00:00, 53.2MB/s]


In [3]:
import zipfile
import os

os.listdir()

for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

In [4]:

# Se lee la base original
base=pd.read_csv('/content/par_esp_interventions_l1_l13.csv')

<ipython-input-4-5462a4a332af>:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  base=pd.read_csv('/content/par_esp_interventions_l1_l13.csv')


In [6]:
import pickle

with open('/content/vocabulary_snowball.voc', 'rb') as f:
    inter = pickle.load(f)

In [7]:
base=base.rename(columns={"party": "partido"})

In [8]:
import nltk

nltk.download('punkt')
from nltk.tokenize import word_tokenize
import string
from nltk.stem.snowball import SpanishStemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [9]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords=stopwords.words('spanish')

# Se incorporan las stopwords sugeridas por Federico
stopwords.extend(['señor', 'señora', 'mucha', 'gracia', 'año', 'pasado', 'cada', 'vez', 'uno', 'dos', 'tres', 'cuatro', 'cinco', 'seis' ,'siete', 'ocho', 'nueve', 'diez'])
stopwords.extend(['hoy', 'aquí', 'primer', 'lugar', 'primera', 'segunda', 'primero', 'segundo', 'siguiente', 'tercer'])
stopwords.extend(['convergencia', 'esquerra', 'republicana', 'grupo', 'parlamentaria', 'parlamentario', 'partido'])
stopwords.extend(['enmienda', 'votación', 'favor', 'abstención', 'ley', 'real', 'decreto', 'decretoley', 'resultado', 'voto', 'sé' ,'sí', 'silencio' ,'favor'])
stopwords.extend(['usted', 'señoría', 'presidente', 'presidenta', 'ministro', 'orden' , 'día', 'palabra', 'petición', 'posición', 'punto', 'vista', 'sesión', 'baldoví', 'duran', 'turno'])
stopwords.extend(['hacer', 'frente', 'puede' ,'ser', 'va', 'voy', 'decir'])
stopwords.extend(['millón', 'euro', 'emitido', 'efectuada', 'dio', 'comienzo', 'partido', 'queda', 'quedan', 'rechazada', 'aceptada', 'comienza', 'usted', 'sabe', 'abstención', 'diputado', 'gobierno'])
stopwords.extend(['continuación', 'votamos', 'telemático', 'republicanaizquierda', 'unidainiciativa', 'puede', 'bien', 'propuesta' ,'abstencion', 'mayoría', 'absoluta', 'pregunta', 'don', 'vamos', 'votar', 'llevar', 'cabo', 'millón', 'muchas', 'gracias'])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
# Elimino de la base las intervenciones del Presidente de la cámara
base=base[(base['role'] != 'PRESIDENTE, PRESIDENTA') | (base['role'] != 'PRESIDENTE DEL CONGRESO DE LOS DIPUTADOS')]

In [11]:
# Eliminos nans de la base
base=base[base['intervention'].notna()]

In [12]:
# Esta celda tarda unos 30 minutos en correr
porter_stemmer = SpanishStemmer()

lista = []
base['tokens']=""
for row in base.values:
    #elimino puntuación
    non_punctuation = row[2].replace('-',' ')
    non_punctuation = non_punctuation.translate(str.maketrans('', '', string.punctuation))
    word_tokens = word_tokenize(non_punctuation)

    #elimino stopwords
    tokens=[w for w in word_tokens if not w.lower() in stopwords]

    #stemming
    stemmers = [porter_stemmer.stem(word) for word in tokens]
    row[18] = [stem for stem in stemmers if stem.isalpha() and len(stem) > 1]
    lista.append(row)

In [13]:
base = pd.DataFrame(lista, columns=base.columns)

In [14]:
base['tokens'] = [str(token) for token in base['tokens']]

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(ngram_range=(2,2) ,vocabulary=inter)
matrix = vect.fit_transform(base['tokens'])

In [16]:
len(vect.vocabulary_)

357780

In [17]:
base.columns

Index(['orden', 'namex', 'intervention', 'fecha', 'legislatura', 'pleno',
       'role', 'oradorx', 'namey', 'partido', 'province', 'congreso', 'senado',
       'total', 'oradory', 'cataluna', 'estatut', 'inter_raw', 'tokens'],
      dtype='object')

In [18]:
# Esta celda tarda unos 70 minutos en correr
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=30, max_iter=50, learning_method='batch', learning_offset=50.,random_state=0)
lda.fit(matrix)

LatentDirichletAllocation(learning_offset=50.0, max_iter=50, n_components=30,
                          random_state=0)

In [19]:
import numpy as np
from numpy import argsort
import statistics
from statistics import mode
from scipy.special import logsumexp
import scipy as sp

# Bases nuevas

In [20]:
disc_weights=matrix*lda.components_.transpose()
disc_weights.shape

(447525, 30)

In [21]:
base[['apellido', 'nombre']] =base.namey.str.split(', ', expand=True)

In [22]:
column_names = [str(i) for i in range(1, 31)]

df_weights = pd.DataFrame(disc_weights, columns=column_names)

select=[ 'fecha', 'apellido', 'nombre' , 'intervention'	, 'partido', 'role']
df_weights = pd.concat([base[select], df_weights], axis=1)

In [23]:
df_weights.to_csv('df_weights.csv')

In [24]:
import scipy as sp
from scipy.special import logsumexp

def frex(mod, w=0.7):
    log_beta = np.log(mod.components_)
    log_exclusivity = log_beta - logsumexp(log_beta, axis=0)
    exclusivity_ecdf = np.apply_along_axis(ecdf, 1, log_exclusivity)
    freq_ecdf = np.apply_along_axis(ecdf, 1, log_beta)
    out = 1. / (w / exclusivity_ecdf + (1 - w) / freq_ecdf)
    return out

def ecdf(arr):
    return sp.stats.rankdata(arr, method='max') / arr.size

In [25]:
frex_df=pd.DataFrame(frex(lda))
disc_frex=matrix*frex_df.transpose()

In [26]:
df_frex = pd.DataFrame(disc_weights, columns=column_names)

select=[ 'fecha', 'apellido', 'nombre' , 'intervention'	, 'partido', 'role']
df_frex = pd.concat([base[select], df_frex], axis=1)


df_frex.to_csv('df_frex.csv')

# Topics por weights

In [27]:
disc_weights=matrix*lda.components_.transpose()

In [28]:
base['Topic_weight']=disc_weights.argmax(axis=1)+1

In [29]:
base.fecha=pd.to_datetime(base.fecha, infer_datetime_format=True)

In [30]:
# Por LEGISLATURA por WEIGHTS
legislaturas=['I', 'II', 'III', 'IV', 'V', 'VI', 'VII','VIII', 'IX', 'X', 'XI', 'XII','XIII']
topics_por_legislatura=pd.DataFrame()
for x in legislaturas:
  base_legislatura=base[base.legislatura==x]
  topics_por_legislatura[x]=base_legislatura['Topic_weight'].value_counts().sort_index()/base_legislatura['Topic_weight'].value_counts().sort_index().sum()
topics_por_legislatura.to_csv('topics_por_legislatura_por_weights.csv')

In [31]:
# Por AÑO por weights
topics_por_año=pd.DataFrame()
for x in range (1979, 2020):
  base_year=base[pd.DatetimeIndex(base.fecha).year==x]
  topics_por_año[x]=base_year['Topic_weight'].value_counts().sort_index()/base_year['Topic_weight'].value_counts().sort_index().sum()
topics_por_año.to_csv('topics_por_año_por_weights.csv')

# FREX

In [32]:
def frex(mod, w=0.7):
    log_beta = np.log(mod.components_)
    log_exclusivity = log_beta - logsumexp(log_beta, axis=0)
    exclusivity_ecdf = np.apply_along_axis(ecdf, 1, log_exclusivity)
    freq_ecdf = np.apply_along_axis(ecdf, 1, log_beta)
    out = 1. / (w / exclusivity_ecdf + (1 - w) / freq_ecdf)
    return out

def ecdf(arr):
    return sp.stats.rankdata(arr, method='max') / arr.size

def t_FREX(row):
  return frex_df[matrix[row['id']].argmax()].argsort()[9]

In [33]:
frex_df=pd.DataFrame(frex(lda))

In [34]:
# Se crea una columna con el topic que se asigna por frex a cada discurso de la base
base['id']=range(0,len(base))
base['Topic_FREX']=base.apply (lambda row: t_FREX(row), axis=1)+1

In [35]:
# Por LEGISLATURA por FREX
legislaturas=['I', 'II', 'III', 'IV', 'V', 'VI', 'VII','VIII', 'IX', 'X', 'XI', 'XII','XIII']
topics_por_legislatura=pd.DataFrame()
for x in legislaturas:
  base_legislatura=base[base.legislatura==x]
  topics_por_legislatura[x]=base_legislatura['Topic_FREX'].value_counts().sort_index()/base_legislatura['Topic_FREX'].value_counts().sort_index().sum()
topics_por_legislatura.to_csv('topics_por_legislatura_por_frex.csv')

In [36]:
# Por AÑO por FREX
topics_por_año=pd.DataFrame()
for x in range (1979, 2020):
  base_year=base[pd.DatetimeIndex(base.fecha).year==x]
  topics_por_año[x]=base_year['Topic_FREX'].value_counts().sort_index()/base_year['Topic_FREX'].value_counts().sort_index().sum()
topics_por_año.to_csv('topics_por_año_por_frex.csv')

# Bigramas por weights

In [37]:
bigramas=pd.DataFrame()
feature_names=vect.get_feature_names_out()
no_top_words=20
for topic_idx, topic in enumerate(lda.components_):
  bigramas["Topic %d:" % (topic_idx + 1)]=[feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]

In [38]:
bigramas.to_csv('bigramas_por_weights_30topics.csv')

# Bigramas por frex

In [39]:
"Topic %d:" % (topic_idx + 1)
for topic_idx, topic in enumerate(frex(lda)):
  bigramas["Topic %d:" % (topic_idx + 1)]=[feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]

In [40]:
bigramas.to_csv('bigramas_por_frex_30topics.csv')

# Bigramas para 10 Topics

In [41]:
# Esta celda tarda unos 25 minutos en correr
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=10, max_iter=50, learning_method='batch', learning_offset=50.,random_state=0)
lda.fit(matrix)

LatentDirichletAllocation(learning_offset=50.0, max_iter=50, random_state=0)

In [42]:
bigramas=pd.DataFrame()
feature_names=vect.get_feature_names_out()
no_top_words=20
for topic_idx, topic in enumerate(lda.components_):
  bigramas["Topic %d:" % (topic_idx + 1)]=[feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]

In [43]:
bigramas.to_csv('bigramas_por_weights_10topics.csv')

In [44]:
"Topic %d:" % (topic_idx + 1)
for topic_idx, topic in enumerate(frex(lda)):
  bigramas["Topic %d:" % (topic_idx + 1)]=[feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]

In [45]:
bigramas.to_csv('bigramas_por_frex_10topics.csv')

# Bajada

In [ ]:
from google.colab import files
files.download('df_weights.csv')
files.download('df_frex.csv')
files.download('topics_por_año_por_weights.csv')
files.download('topics_por_legislatura_por_weights.csv')
files.download('topics_por_año_por_frex.csv')
files.download('topics_por_legislatura_por_frex.csv')
files.download('bigramas_por_weights_30topics.csv')
files.download('bigramas_por_frex_30topics.csv')
files.download('bigramas_por_frex_10topics.csv')